In [1]:
import time
import pandas as pd
from tqdm.notebook import tqdm
from selenium import webdriver
import numpy as np 
from pathlib import Path
from unidecode import unidecode
import os 

path = 'google_scholar'

root_folder = Path(path)
root_folder.mkdir(parents=True, exist_ok=True)

In [2]:
df_searchterms = pd.read_csv('searchterms.csv')
df_seq = pd.DataFrame(np.arange(0,10000,10), columns = ['start'])

df_searchterms['k']=1
df_seq['k']=1

df_searchterms = df_searchterms.merge(df_seq, on ='k').drop(columns = 'k')
df_searchterms['searchterm'] = df_searchterms['searchterm'].map(lambda x: unidecode(x)).str.lower().str.strip()
df_searchterms['folder'] = df_searchterms['searchterm'].str.replace(' ','_',regex = False)
df_searchterms['url'] = "https://scholar.google.com.br/scholar?start="+ df_searchterms['start'].astype('str')+"&q="+df_searchterms['searchterm'].str.replace(' ','+',regex = False)+"&hl=pt-BR&as_sdt=0,5"
df_searchterms['file'] = (
    df_searchterms['url']
    .str.replace(r'[^a-zA-Z0-9_]', '_', regex=True)  # substitui caracteres especiais por _
    .str.replace(r'_+', '_', regex=True)             # múltiplos _ por 1 _
    .str.strip('_')                                  # remove _ no começo/fim
    + '.html'
)

#LIMITE PARA TESTES
df_searchterms = df_searchterms[df_searchterms['start'] < 200].copy()

In [3]:
df_searchterms.to_csv(path+'/google_scholar_search_base.csv', sep = ';', index = False)

In [4]:

# Inicializa o navegador
driver = webdriver.Chrome()
url = f"https://scholar.google.com.br/scholar?start=0&q=visualization+machine+learning&hl=pt-BR&as_sdt=0,5"
driver.get(url)



In [5]:
records_search = df_searchterms.to_dict(orient = 'records')
records_search_filter = []
for record in records_search:
    local_folder = caminho_completo = root_folder /'raw_data' / record['folder']
    local_folder.mkdir(parents=True, exist_ok=True)

    local_file = local_folder / record['file']
    if local_file.exists() and local_file.is_file():  
        continue

    records_search_filter.append((record, local_file))

for record, local_file in tqdm(records_search_filter):

    driver.get(record['url'])
    time.sleep(2)
    alpha, beta = 3,6
    val = np.random.beta(alpha, beta)
    wait_time = 1 + val * (3 - 1)
    time.sleep(wait_time) 

    with open(local_file, "w", encoding="utf-8") as f:
        f.write(driver.page_source)

    

  0%|          | 0/145 [00:00<?, ?it/s]

In [6]:
#for i in range(len(records_search_filter)):
#    record, local_file = records_search_filter[i]
#    local_file.unlink()